In [1]:
#Building a regression model that tries to predict the estimated cumulative 6 month revenue from a new customer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import datetime as dt
%matplotlib inline
import seaborn as sns
import lifetimes

In [2]:
item = pd.read_csv('olist_order_items_dataset.csv')
customer = pd.read_csv('olist_customers_dataset.csv')
payment = pd.read_csv('olist_order_payments_dataset.csv')
review = pd.read_csv('olist_order_reviews_dataset.csv')	
order = pd.read_csv('olist_orders_dataset.csv')
product = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

In [3]:
#order_id, order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value
cols = ['order_id','product_id', 'seller_id', 'price', 'freight_value']
item = item[cols]
item = item.set_index('order_id')
item


,product_id,seller_id,price,freight_value
order_id,,,,
00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14
...,...,...,...,...
fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,299.99,43.41
fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,350.00,36.53
fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,99.90,16.95


In [4]:
item.shape

(112650, 4)

In [5]:
item[('seller_id')].describe()

count                               112650
unique                                3095
top       6560211a19b47992c3666cc44a7e94c0
freq                                  2033
Name: seller_id, dtype: object

In [6]:
item[('product_id')].describe()

count                               112650
unique                               32951
top       aca2eb7d00ea1a7b8ebd4e68314663af
freq                                   527
Name: product_id, dtype: object

In [7]:
cols =['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
payment = payment[cols]

payment

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [9]:
payment.shape

(103886, 5)

In [10]:
payment= payment.groupby('order_id').sum()
payment

,payment_sequential,payment_installments,payment_value
order_id,,,
00010242fe8c5a6d1ba2dd792cb16214,1,2,72.19
00018f77f2f0320c557190d7a144bdd3,1,3,259.83
000229ec398224ef6ca0657da4fc703e,1,5,216.87
00024acbcdf0a6daa1e931b038114c75,1,2,25.78
00042b26cf59d7ce69dfabb4e55b4fd9,1,3,218.04
...,...,...,...
fffc94f6ce00a00581880bf54a75a037,1,1,343.40
fffcd46ef2263f404302a634eb57f7eb,1,1,386.53
fffce4705a9662cd70adb13d4a31832d,1,3,116.85


In [11]:
customer = pd.read_csv('olist_customers_dataset.csv')
cols = ['customer_id','customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
customer = customer.set_index('customer_id')
customer.head()


,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
customer_id,,,,
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [12]:
customer.shape

(99441, 4)

In [14]:
customer['customer_city'].value_counts()

sao paulo         15540
rio de janeiro     6882
belo horizonte     2773
brasilia           2131
curitiba           1521
                  ...  
agua doce             1
tururu                1
boquira               1
picada cafe           1
dom cavati            1
Name: customer_city, Length: 4119, dtype: int64

In [ ]:
#order table 
cols = ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
order = order[cols]
order = order.set_index('order_id')
order.head()

In [ ]:
order.shape

In [ ]:
cols = ['order_id','review_score', 'review_creation_date', 'review_answer_timestamp']
review1 = review[cols]
review1 = review.set_index('order_id')
review1

In [ ]:
review = review.sort_values(by='order_id', ascending=False, na_position='first')
review= review.groupby('order_id').mean()
review

In [ ]:
#Joining tables
result = pd.merge(item, payment,how='left', on= 'order_id').reset_index()
result = result.sort_values(by='order_id')
result=result.drop_duplicates(subset=None, keep= 'first', inplace=False)
result.shape

In [ ]:
result.head()

In [ ]:
#Joining tables
#item, payment, product, order, customer, review
result = pd.merge(result, product,how='inner', on= 'product_id').sort_values(by='order_id')

In [ ]:
result.shape

In [ ]:
result = pd.merge(result, order,how='inner', on= 'order_id')
result = result.sort_values(by='order_id')
result = result.loc[:,~result.columns.duplicated()]
#result.columns

In [ ]:
result1= result.loc[:,~result.columns.duplicated()]
result1.set_index('customer_id')




In [ ]:
customer.head()


In [ ]:
result1 = pd.merge(result1, customer,how='inner', on= 'customer_id').set_index('customer_id')
result1

In [ ]:
result1.columns

In [ ]:
result1.drop(columns=['customer_unique_id_y', 'customer_zip_code_prefix_y', 'customer_city_y',
       'customer_state_y', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'])





In [ ]:
counts = result1[('customer_unique_id')].value_counts()
count=result1.drop_duplicates()
counts

In [ ]:
fig = plt.figure(figsize=(10,10)) # define plot area
ax = fig.gca() # define axis    
counts.plot.bar(ax = ax) # Use the plot.bar method on the counts data frame
ax.set_title('Number of products by type') # Give the plot a main title
ax.set_xlabel('Product_category_name') # Set text for the x axis
ax.set_ylabel('Number of items')# Set text for y axis

In [ ]:
result1.columns

In [ ]:
result1['order_purchase_timestamp'] = pd.to_datetime(result1['order_purchase_timestamp'])
result1['order_date'] = result1.order_purchase_timestamp.dt.date
result1['order_delivered_customer_date'] = pd.to_datetime(result1['order_delivered_customer_date'])
result1['order_estimated_delivery_date'] = pd.to_datetime(result1['order_estimated_delivery_date'])
result1['order_approved_at'] = pd.to_datetime(result1['order_approved_at'])

In [ ]:
result1['order_purchase_timestamp'].describe

In [ ]:
result1['customer_unique_id'].describe

In [ ]:
DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00', '2018-01-01 03:00:00',
               '2018-01-01 04:00:00', '2018-01-01 05:00:00',
               '2018-01-01 06:00:00', '2018-01-01 07:00:00',
               '2018-01-01 08:00:00', '2018-01-01 09:00:00',
               ...
               '2018-01-07 15:00:00', '2018-01-07 16:00:00',
               '2018-01-07 17:00:00', '2018-01-07 18:00:00',
               '2018-01-07 19:00:00', '2018-01-07 20:00:00',
               '2018-01-07 21:00:00', '2018-01-07 22:00:00',
               '2018-01-07 23:00:00', '2018-01-08 00:00:00'],
              dtype='datetime64[ns]', length=169, freq='H')

In [ ]:

result1['order_date'] = pd.order_purchase_timestamp.dt.date
result1['order_date'] = pd.to_datetime(result1['order_date'])

#cols = ['customer_unique_id', 'order_date']
#elog = elog[cols]

#elog.columns = ['CUSTOMER_ID', 'ORDER_DATE']


#result1.info()
display(result1.sample(5))